In [4]:
import torch
import data_loader.data_loaders as module_data
from data_loader.data_loaders import CollectDataLoader
import model.model as module_arch
from utils import get_instance

import math
import librosa
from IPython.display import display, Audio
from tqdm import tqdm
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

ImportError: cannot import name 'get_instance' from 'utils' (/Users/Marco/Documents/GOOGLE DRIVE/PHD/REPOS/vae-audio/utils/__init__.py)

In [1]:
model_path = 'saved/models/Conv1dSpecVAE/0923_153845/model_best.pth'
sr = 22050
n_fft = 2048
hop_length = 512

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resume = torch.load(model_path, map_location=device)

NameError: name 'torch' is not defined